# JSON Schema generation

A simple usage of the library that, given generates a JSON Schema for inputs and outputs.

## 1. Parsing

In this sample we'll show the access from a remote public URL.

In [1]:
from cwl_loader import load_cwl_from_location
from cwl2ogc import BaseCWLtypes2OGCConverter

cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/cwl2ogc/refs/heads/develop/tests/artifacts/cwl-types/array-inputs.cwl')
cwl_converter = BaseCWLtypes2OGCConverter(cwl_document)

2025-08-11 19:05:30.506 | INFO     | cwl_loader:load_cwl_from_location:186 - Loading CWL document from https://raw.githubusercontent.com/eoap/cwl2ogc/refs/heads/develop/tests/artifacts/cwl-types/array-inputs.cwl...
2025-08-11 19:05:30.705 | INFO     | cwl_loader:load_cwl_from_yaml:117 - Updating the model to v1.2...
2025-08-11 19:05:30.709 | INFO     | cwl_loader:load_cwl_from_yaml:128 - Raw CWL document successfully updated to v1.2! Now converting to the CWL model...
2025-08-11 19:05:31.105 | INFO     | cwl_loader:load_cwl_from_yaml:136 - Raw CWL document successfully updated to v1.2! Now dereferencing the FQNs...
2025-08-11 19:05:31.105 | INFO     | cwl_loader:_clean_process:62 -   Cleaning CommandLineTool array-inputs...
2025-08-11 19:05:31.106 | INFO     | cwl_loader:load_cwl_from_yaml:144 - CWL document successfully dereferenced!


## 2. Inputs JSON Schema generation

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the JSON schema:

In [2]:
import sys

cwl_converter.dump_inputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/inputs.yaml",
  "description": "The schema to represent a array-inputs inputs definition",
  "type": "object",
  "required": [
    "filesA",
    "filesB",
    "filesC"
  ],
  "properties": {
    "filesA": {
      "$ref": "#/$defs/filesA"
    },
    "filesB": {
      "$ref": "#/$defs/filesB"
    },
    "filesC": {
      "$ref": "#/$defs/filesC"
    }
  },
  "additionalProperties": false,
  "$defs": {
    "filesA": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesB": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesC": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}

### 2.1 Inputs validation

Schema can be used to fully validate an inputs dictionary (expecting JSON Schema validation errors in the example below):

In [3]:
from jsonschema import Draft202012Validator
from jsonschema.exceptions import SchemaError
import json

inputs = {
    "aoi": "-118.985,38.432,-118.183,38.938",
    "filesB": "EPSG:4326",
    "bands": [ "green", "nir08" ],
    "item": {
        "class":"https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL",
        "value":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
    }
}

try:
    errors_list = list(
        map(
            lambda error: {
                "type": '.'.join(error.schema_path),
                "title": '.'.join(error.path),
                "instance": error.instance,
                "detail": error.message
            },
            Draft202012Validator(cwl_converter.get_inputs_json_schema()).iter_errors(inputs)
        )
    )

    if errors_list:
        json.dump(errors_list, sys.stdout, indent=2)
except SchemaError as schema_error:
    json.dump({
        "type": "JSONSchemaError",
        "title": "CWL2OGC generated JSON Schema error",
        "detail": schema_error.message
    }, sys.stdout, indent=2)

[
  {
    "type": "required",
    "title": "",
    "instance": {
      "aoi": "-118.985,38.432,-118.183,38.938",
      "filesB": "EPSG:4326",
      "bands": [
        "green",
        "nir08"
      ],
      "item": {
        "class": "https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL",
        "value": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
      }
    },
    "detail": "'filesA' is a required property"
  },
  {
    "type": "required",
    "title": "",
    "instance": {
      "aoi": "-118.985,38.432,-118.183,38.938",
      "filesB": "EPSG:4326",
      "bands": [
        "green",
        "nir08"
      ],
      "item": {
        "class": "https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL",
        "value": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
      }
    },
    "detail": "'filesC' is a required propert

## 3. Outputs JSON Schema generation

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the JSON Schema:

In [4]:
cwl_converter.dump_outputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/outputs.yaml",
  "description": "The schema to represent a array-inputs outputs definition",
  "type": "object",
  "required": [
    "example_out"
  ],
  "properties": {
    "example_out": {
      "$ref": "#/$defs/example_out"
    }
  },
  "additionalProperties": false,
  "$defs": {
    "example_out": {
      "type": "string"
    }
  }
}

### 2.1 Outputs validation

Schema can be used to fully validate an outputs dictionary (JSON Schema validation expected to pass):

In [5]:
outputs = {
    "example_out": "In girum imus nocte et consumimur igni"
}

errors_list = list(
    map(
        lambda error: {
            "type": '.'.join(error.schema_path),
            "title": '.'.join(error.path),
            "instance": error.instance,
            "detail": error.message
        },
        Draft202012Validator(cwl_converter.get_outputs_json_schema()).iter_errors(outputs)
    )
)

if not errors_list:
    print("Outputs are valid!")

Outputs are valid!
